In [1]:
import time
import datetime
import urllib.request, json
import requests

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from bs4 import BeautifulSoup

from oceanoobsbrasil.db import GetData
%load_ext autoreload
%autoreload 2

In [20]:
df = pd.read_csv('../oceanoobsbrasil/data/tide_table.csv', delimiter='\t')

In [21]:
GetData().feed_bd(table='stations', df=df)

      id                                   name        lat        lon  \
0   1159                        PORTO DE MACEIÓ  -9.683333 -35.718056   
1   1160              BARRA NORTE - ARCO LAMOSO   1.433611 -49.217500   
2   1161                       PORTO DE SANTANA  -0.051944 -51.166944   
3   1162                IGARAPÉ GRANDE DO CURUÁ   0.752222 -50.116667   
4   1163            PORTO DE ARATU - BASE NAVAL -12.796667 -38.491667   
5   1164                      PORTO DE SALVADOR -12.952500 -38.516667   
6   1165                 PORTO DE MADRE DE DEUS -12.735278 -38.617500   
7   1166              PORTO DE ILHÉUS - MALHADO -14.768889 -39.018333   
8   1167            TERMINAL PORTUÁRIO DO PECÉM  -3.533611 -38.785833   
9   1168          PORTO DE MUCURIPE - FORTALEZA  -3.702500 -38.468333   
10  1169                       PORTO DE TUBARÃO -20.284167 -40.235000   
11  1170               TERMINAL DA PONTA DO UBU -20.783889 -40.567222   
12  1171                       ILHA DA TRINDADE -20

In [2]:
url = 'https://www.marinha.mil.br/chm/tabuas-de-mare'
resp = requests.get(url)
soup = BeautifulSoup(resp.text,'html.parser')

In [11]:
lines1 = []
lines = soup.find_all('td', attrs={'class': 'views-field-field-ficha-mare'})
for line in lines:
    lines1.append(line.a['href'])

In [12]:
lines1

['https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/26-porto_de_maceio.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/1-barra_norte_arco_lamoso.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/3-porto_de_santana_-_cia_docas_de_santana.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/2-igarape_grande_do_curua_0.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/30-porto_de_aratu.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/31-porto_de_salvador.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/29-porto_madre_de_deus.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/32-porto_de_ilheus.pdf',
 'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/16-terminal_portua

In [10]:
lines[0].a['href']

'https://www.marinha.mil.br/chm/sites/www.marinha.mil.br.chm/files/dados_de_mare/26-porto_de_maceio.pdf'

In [4]:
from oceanoobsbrasil.others.ndbc import Ndbc

In [5]:
n = Ndbc()

In [7]:
n.get()

/mnt/code/code/soutobias/oceanoobsbrasil/oceanoobsbrasil/others/ndbc.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.result.wspd[self.result.wspd.notnull()] = (
/mnt/code/code/soutobias/oceanoobsbrasil/oceanoobsbrasil/others/ndbc.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.result.wspd[self.result.wspd.notnull()] = (


(904, 14)


In [109]:
n.result[(n.result.lat>-40)  & (n.result.lat < 15) & (n.result.lon < -8) & (n.result.lon > -60)]

,name,date_time,lat,lon,wdir,wspd,swvht,tp,pres,atmp,sst,dewpt,swvht_swell,wvdir_swell,institution,data_type
18,SHIP,2023-09-05,-34.9,-56.2,NaN,NaN,NaN,NaN,1020.2,9.0,NaN,4.7,NaN,NaN,ndbc,gts
63,SHIP,2023-09-05,-32.8,-14.2,310.0,6.0,0.5,3.0,1035.5,18.0,18.3,8.2,1.5,190.0,ndbc,gts
104,SHIP,2023-09-05,12.0,-17.8,150.0,6.0,0.5,4.0,1010.4,25.8,NaN,23.1,NaN,NaN,ndbc,gts
169,SHIP,2023-09-05,-16.9,-14.3,110.0,29.9,4.0,8.0,1024.6,19.2,22.9,13.4,3.5,140.0,ndbc,gts
275,SHIP,2023-09-05,6.4,-27.9,NaN,NaN,NaN,NaN,1016.1,27.3,NaN,26.6,NaN,NaN,ndbc,gts
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4413,SHIP,2023-09-05,-33.9,-43.0,10.0,28.0,1.5,8.0,1013.0,19.0,18.0,17.4,2.5,80.0,ndbc,gts
4416,SHIP,2023-09-05,-32.5,-16.9,350.0,2.9,0.5,4.0,1035.0,21.0,19.0,12.1,1.5,180.0,ndbc,gts
4444,SHIP,2023-09-05,-26.1,-40.9,30.0,28.0,NaN,NaN,1016.4,23.0,NaN,19.9,NaN,NaN,ndbc,gts
4463,SHIP,2023-09-05,12.8,-25.1,NaN,NaN,NaN,NaN,1011.1,30.6,NaN,26.4,NaN,NaN,ndbc,gts


In [110]:
n.result = n.result[(n.result.lat>-40)  & (n.result.lat < 15) & (n.result.lon < -8) & (n.result.lon > -60)]

In [32]:
url = f"https://www.ndbc.noaa.gov/ship_obs.php?uom=M&time=12"

In [67]:
len(data[0])

24

In [68]:
len(new_header)

40

In [91]:
resp = requests.get(url)

soup = BeautifulSoup(resp.text, "html.parser")

lines = soup.find_all("pre")
count = 0
data = []
first_loop = True
for line in lines:
    inner_lines = line.find_all('span')
    first_element = 1
    for inner_line in inner_lines:
        if first_element:
            first_element = 0
            new_header = inner_line.get_text(strip=True)
            new_header = new_header.replace("B", " B")
            new_header = re.sub(" +", ",", new_header).split(",")
        else:
            new_data = inner_line.get_text(strip=True)
            new_data = new_data.replace("B", " B")
            new_data = re.sub(" +", ",", new_data).split(",")
            data.append(new_data)
    if first_loop:
        first_loop = False
        all_array = pd.DataFrame(data)
    else:
        array = pd.DataFrame(data)

        all_array = pd.concat([all_array, array])
        
all_array.columns = new_header[:26]




In [96]:
all_array.columns

Index(['SHIP', 'HOUR', 'LAT', 'LON', 'WDIR', 'WSPD', 'GST', 'WVHT', 'DPD',
       'PRES', 'PTDY', 'ATMP', 'WTMP', 'DEWP', 'VIS', 'TCC', 'S1HT', 'S1PD',
       'S1DIR', 'S2HT', 'S2PD', 'S2DIRIceSeaID', '(GMT)', '°T', 'm/s', 'm/s'],
      dtype='object')

In [85]:
new_header

['SHIP',
 'HOUR',
 'LAT',
 'LON',
 'WDIR',
 'WSPD',
 'GST',
 'WVHT',
 'DPD',
 'PRES',
 'PTDY',
 'ATMP',
 'WTMP',
 'DEWP',
 'VIS',
 'TCC',
 'S1HT',
 'S1PD',
 'S1DIR',
 'S2HT',
 'S2PD',
 'S2DIRIceSeaID',
 '(GMT)',
 '°T',
 'm/s',
 'm/s',
 'm',
 'sec',
 'mb',
 'mb',
 '°C',
 '°C',
 '°C',
 'km',
 '8thm/s',
 'sec',
 '°T',
 'm/s',
 'sec',
 '°TAccIce-------------------------------------------------------------------------------------------------------------------------------------------------']

In [87]:
new_header[:25]

['SHIP',
 'HOUR',
 'LAT',
 'LON',
 'WDIR',
 'WSPD',
 'GST',
 'WVHT',
 'DPD',
 'PRES',
 'PTDY',
 'ATMP',
 'WTMP',
 'DEWP',
 'VIS',
 'TCC',
 'S1HT',
 'S1PD',
 'S1DIR',
 'S2HT',
 'S2PD',
 'S2DIRIceSeaID',
 '(GMT)',
 '°T',
 'm/s']

In [89]:
    all_array.columns = new_header[:26]

In [90]:
all_array

,SHIP,HOUR,LAT,LON,WDIR,WSPD,GST,WVHT,DPD,PRES,...,S1HT,S1PD,S1DIR,S2HT,S2PD,S2DIRIceSeaID,(GMT),°T,m/s,m/s
0,SHIP,00,47.6,-52.6,140,2.1,-,-,-,1014.1,...,-,-,-,-,-,-,----,-----,None,NaN
1,SHIP,00,62.7,-50.8,130,7.0,-,-,-,999.3,...,-,-,-,-,-,-,----,-----,None,NaN
2,SHIP,00,48.6,-123.4,130,4.6,-,-,-,1017.3,...,-,-,-,-,-,-,----,-----,None,NaN
3,SHIP,00,44.7,-86.5,180,7.7,-,0.5,-,1018.4,...,-,-,-,-,-,-,----,-----,None,NaN
4,SHIP,00,-14.5,125.4,-,-,-,-,-,1015.8,...,-,-,-,-,-,-,----,-----,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4463,SHIP,12,37.0,-73.4,300,8.2,-,1.5,8.0,1026.9,...,-,-,-,-,-,-,----,-----,None,None
4464,SHIP,12,44.0,-76.5,280,3.6,-,-,-,-,...,-,-,-,-,-,-,----,-----,None,None
4465,SHIP,12,43.1,-82.4,200,6.2,-,-,-,-,...,-,-,-,-,-,-,----,-----,None,None
4466,SHIP,12,43.0,-82.4,220,6.7,-,-,-,-,...,-,-,-,-,-,-,----,-----,None,None


In [74]:
header[0][:24]

['SHIP',
 'HOUR',
 'LAT',
 'LON',
 'WDIR',
 'WSPD',
 'GST',
 'WVHT',
 'DPD',
 'PRES',
 'PTDY',
 'ATMP',
 'WTMP',
 'DEWP',
 'VIS',
 'TCC',
 'S1HT',
 'S1PD',
 'S1DIR',
 'S2HT',
 'S2PD',
 'S2DIRIceSeaID',
 '(GMT)',
 '°T']

In [71]:
len(data[0])

24

In [48]:
pd.DataFrame(np.array(data))

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (4353,) + inhomogeneous part.

In [42]:
x = inner_lines[1]
line = x.get_text(strip=True)
line = line.replace("B", " B")
line = re.sub(" +", ",", line).split(",")
line

['SHIP',
 '12',
 '49.2',
 '-53.2',
 '200',
 '10.3',
 '-',
 '-',
 '-',
 '1013.9',
 '+1.0',
 '13.9',
 '-',
 '15.3',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '-',
 '----',
 '-----']

In [6]:
resp = requests.get(n.url)

In [7]:
soup = BeautifulSoup(resp.text, "html.parser")

In [9]:
import pandas as pd

In [14]:
lines = soup.find_all("pre")

In [22]:
lines[1].find_all('span')[1]

<span>SHIP     23  25.8  -79.8    -     -     -     -     - 1013.0  -0.8  32.8     -  24.9     -    -     -     -      -     -     -      -  ---- -----</span>

In [24]:
import re

In [26]:
lines[1].find_all('span')[1]
for line in lines:
    line = line.get_text(strip=True)
    line = line.replace("B", " B")
    line = re.sub(" +", ",", line).split(",")
#     values.append(line)

In [31]:
lines[0]

<pre class="wide-content"><span class="obshdr">SHIP    HOUR  LAT    LON WDIR  WSPD   GST  WVHT   DPD   PRES  PTDY  ATMP  WTMP  DEWP   VIS  TCC  S1HT  S1PD  S1DIR  S2HT  S2PD  S2DIR   <a href="javascript:show_description('ice_accre')" title="Show Ice Accretion Description">Ice</a>   <a href="javascript:show_description('sea_ice')" title="Show Sea Ice Description">Sea</a>
 ID    (GMT)               °T   m/s   m/s     m   sec     mb    mb    °C    °C    °C    km  8<sup>th</sup>   m/s   sec     °T    m/s   sec    °T   <a href="javascript:show_description('ice_accre')" title="Show Ice Accretion Description">Acc</a>   <a href="javascript:show_description('sea_ice')" title="Show Sea Ice Description">Ice</a>
-------------------------------------------------------------------------------------------------------------------------------------------------</span>
<span>SHIP     00  36.9  -76.3  120   2.6     -     -     - 1013.3     -  31.8  27.0  21.9     -    -     -     -      -     -     -     

In [12]:
pd.read_html(resp.text)

ValueError: No tables found

In [11]:
n.get()

https://www.ndbc.noaa.gov/radial_search.php?lat1=-15&lon1=-18&uom=M&dist=4000&ot=A&time=12


ValueError: Length mismatch: Expected axis has 0 elements, new values have 26 elements

In [12]:
n.soup


<!DOCTYPE html>

<html lang="en">
<head>
<title>NDBC - Observations - Radial Search</title>
<meta content="width=device-width, initial-scale=1.0" name="viewport"/>
<meta content="Text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="National Data Buoy Center" name="DC.title"/>
<meta content="Provides nearby observations for specified location, range and time frame." name="DC.description"/>
<meta content="weather, buoy, weather buoy, marine, forecast, hurricane, wind, wave, offshore, surfing, temperature, meteorology, climate, ocean" name="DC.subject"/>
<meta content="US Department of Commerce, National Oceanic and Atmospheric Administration, National Weather Service, National Data Buoy Center" name="DC.creator"/>
<meta content="EN-US" name="DC.language" scheme="DCTERMS.RFC1766"/>
<meta content="text/html;charset=iso-8859-1" name="DC.format"/>
<meta content="1996-11-08" name="DC.date.created" scheme="ISO8601"/>
<link href="/style/ndbc2022a.css" rel="stylesheet" type="tex

In [ ]:
https://www.ndbc.noaa.gov/box_search.php?lat1=-35.8333&lat2=7&lon1=-55.2&lon2=20&uom=M&ot=A&time=12

In [7]:
n.x

,ID,T1,hour,LAT,LON,WDIR,WSPD,GST,WVHT,DPD,...,VIS,TCC,TIDE,S1HT,S1PD,S1DIR,S2HT,S2PD,S2DIR,date_time
0,SHIP,S,11,5.00,-10.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-30 11:00:00
1,SHIP,S,11,-7.90,-14.40,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-30 11:00:00
2,SHIP,S,11,-24.20,-46.20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-30 11:00:00
3,SHIP,S,10,1.60,-45.60,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-30 10:00:00
4,SHIP,S,10,-23.90,-46.20,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-30 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
158,31007,B,23,0.00,-23.00,105,6.7,7.6,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-29 23:00:00
159,15006,B,23,-6.00,-10.00,129,6.0,7.4,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-29 23:00:00
160,31003,B,23,-8.00,-30.00,111,8.6,10.2,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-29 23:00:00
161,31005,B,23,-19.00,-34.00,57,6.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-07-29 23:00:00


In [31]:
n.result.wspd = pd.to_numeric(n.result.wspd, errors='coerce')
n.result.wspd.iloc[0]

5.8

In [30]:
n.result.wspd[n.result.wspd.notnull()]

0       5.8
1       4.1
3       4.6
4       9.8
6       7.2
       ... 
122     6.7
123     9.3
124     7.0
125     9.3
126    13.9
Name: wspd, Length: 93, dtype: object

In [18]:
n.get()

TypeError: can't multiply sequence by non-int of type 'float'

In [10]:
db = Getdata()
df = db.get(table='data_no_stations', institution=['in', ['ndbc']], date_time=['>', '2021-09-12'])
df.shape

(139, 17)

In [11]:
type(df.shape)

tuple